In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.models import model_from_json
import numpy as np
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.optimizers import SGD

from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score

from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

import os
import keras
print (keras.__version__)

import cv2

Using TensorFlow backend.


2.2.4


In [2]:
# Doing some configuration
img_width, img_height = 100,100
# Dir for train and test
train_data_dir = 'data_reduced/train'
validation_data_dir = 'data_reduced/test'
nb_train_samples = 386
nb_validation_samples = 155
epochs = 1
running_times = 1
batch_size = (nb_train_samples // epochs) * running_times
print (batch_size)
input_shape = None
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)    

386


In [3]:
generator = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0.,
    width_shift_range=0.,
    height_shift_range=0.,
    shear_range=0.,
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=K.image_data_format())

train_generator = generator.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=nb_train_samples,
    class_mode='binary')

validation_generator = generator.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=nb_validation_samples,
    class_mode='binary')

Found 386 images belonging to 2 classes.
Found 155 images belonging to 2 classes.


In [7]:
class MyClassifier:
    def __init__(self):
        self.vgg16 = keras.applications.vgg16.VGG16(include_top=False,
                                                    weights='imagenet',
                                                    input_shape=input_shape)
        self.tree = XGBClassifier(max_depth=20, 
                                  learning_rate=0.001,                                           
                                  n_estimators=2000,
                                  nthread=8)
        
        self.svc = SVC(probability=True)
        self.sgd = SGDClassifier(loss='log')
        self.ensemble = VotingClassifier(estimators=[
            ('svm', self.svc),
            ('sgd', self.sgd),
            ('xgboost', self.tree)
        ], voting='soft')
    
    def __preproc__(self, X):
#         X = X / 255.
        X = self.vgg16.predict(X)
        X = X.reshape([len(X), -1])
        return X
        
    def fit(self, X, y):
        X = self.__preproc__(X)
        if X is not None:
            self.tree.fit(X, y)
        
    def predict(self, X):
        X = self.__preproc__(X)
        return self.tree.predict(X)
    
    def predict_proba(self, X):
        X = self.__preproc__(X)
        return self.tree.predict_proba(X)
    
clf = MyClassifier()

In [8]:
X_test, y_test = validation_generator.next()

In [9]:
X, y = train_generator.next()
clf.fit(X, y)
accuracy = accuracy_score(y_test, clf.predict(X_test))
print (accuracy)

0.535483870967742
